## Data Collection

In [1]:
import json
import pandas as pd
import os
import csv
import synapseclient
from synapseclient import Project, File, Folder, Entity, Link
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
from synapseclient import Activity, Wiki
from synapseclient import Evaluation, Submission, SubmissionStatus

connecting to Synapse.org

In [3]:
email = '***************'
password ='sMeuN8fNJfMC4Kw'
syn = synapseclient.Synapse()
syn.login(email,password)

Welcome, tomer zamir!



In [4]:
project = syn.get('syn4993293')

# downloading the data

In [11]:
import tqdm
folder=r'C:\Machine Learning\vp-ws\mPower\voice-data'

In [8]:
for offset in range(0,65000,1000):
    results = syn.tableQuery('SELECT * FROM syn5511444 LIMIT 1000 OFFSET '+str(offset))
    file_map = syn.downloadTableColumns(results,['audio_audio.m4a'],folder, verbose=True)

downloading the table as csv

In [55]:
# Query the table and download the data as a dataframe
table_id = 'syn5511444'
query = 'SELECT * FROM %s' % table_id
df = syn.tableQuery(query).asDataFrame()


In [44]:
# Download the entire table as a file
table_file = syn.downloadTable(table_id)

# Load the data from the file into a Pandas dataframe
df = pd.read_csv(table_file)

# Save the dataframe to a CSV file
df.to_csv('table.csv', index=False)


,recordId,healthCode,createdOn,appVersion,phoneInfo,audio_audio.m4a,audio_countdown.m4a,medTimepoint
72137_9,177d6c17-6635-4884-8481-a671fca06266,639e8a78-3631-4231-bda1-c911c1b169e5,1425904701000,"version 1.0, build 7",iPhone 6,5404521,5404533,I don't take Parkinson medications
72138_9,d59f922b-3ab9-4105-be30-4c2c481e99ed,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,1425926292000,"version 1.0, build 7",iPhone 5s (GSM),5410381,5410398,I don't take Parkinson medications
72139_9,0e85415c-75ee-45d2-8d38-ac7b2c8c5c50,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1425926706000,"version 1.0, build 7",iPhone 6 Plus,5390859,5390881,I don't take Parkinson medications
72140_9,40a9e3a9-1518-4f3f-a9a3-0715233c42a7,45b4e2ca-8d15-4736-828c-829e3d4177f4,1425926937000,"version 1.0, build 7",iPhone 6,5701183,5701251,I don't take Parkinson medications
72141_9,e3b8f800-2c0d-4b42-ba44-f940464dd7b5,45b4e2ca-8d15-4736-828c-829e3d4177f4,1425926960000,"version 1.0, build 7",iPhone 6,5750379,5750420,I don't take Parkinson medications


# mapping the data

downloading a map to sync between file handles in the csv and file names

In [117]:
# QUERY THE mPower PROJECT (syn4993293) FOR ALL OF THE TABLES
tables = syn.getChildren('syn4993293', ['table'])
tables = [t for t in tables if t['name'].startswith('Voice')]

allData = {table['name']: syn.tableQuery('SELECT * FROM %s' % table['id']) for table in tables}
#print(allData)
df = allData['Voice Activity'].asDataFrame()
print(df.head(2))

# return a python dictionary where the keys are the filehandleIds stored in the downloaded column
voiceMap = syn.downloadTableColumns(allData['Voice Activity'], "audio_audio.m4a")

                                     recordId  \
72137_9  177d6c17-6635-4884-8481-a671fca06266   
72138_9  d59f922b-3ab9-4105-be30-4c2c481e99ed   

                                   healthCode      createdOn  \
72137_9  639e8a78-3631-4231-bda1-c911c1b169e5  1425904701000   
72138_9  52fe366a-2a9f-4260-9fb1-0fbc637a6cf4  1425926292000   

                   appVersion        phoneInfo  audio_audio.m4a  \
72137_9  version 1.0, build 7         iPhone 6          5404521   
72138_9  version 1.0, build 7  iPhone 5s (GSM)          5410381   

         audio_countdown.m4a                        medTimepoint  
72137_9              5404533  I don't take Parkinson medications  
72138_9              5410398  I don't take Parkinson medications  


In [118]:
print('voiceMap type: ',type(voiceMap))
print(print('voiceMap type: ',len(voiceMap)))

voiceMap type:  <class 'collections.OrderedDict'>
voiceMap type:  65022
None


In [172]:
df = pd.DataFrame()

# Extract the keys and values from voiceMap
keys = list(voiceMap.keys())
values = list(voiceMap.values())

# Add the keys and values to the DataFrame as separate columns
df['file_handle'] = keys
df['file_name'] = values

df

,file_handle,file_name
0,5404521,c:/users/tomer/.synapsecache/521/5404521/audio...
1,5410381,c:/users/tomer/.synapsecache/381/5410381/audio...
2,5390859,c:/users/tomer/.synapsecache/859/5390859/audio...
3,5701183,c:/users/tomer/.synapsecache/183/5701183/audio...
4,5750379,c:/users/tomer/.synapsecache/379/5750379/audio...
...,...,...
65017,6092288,C:\Users\tomer\.synapseCache\288\6092288\audio...
65018,6092590,C:\Users\tomer\.synapseCache\590\6092590\audio...
65019,6091758,C:\Users\tomer\.synapseCache\758\6091758\audio...
65020,6092458,C:\Users\tomer\.synapseCache\458\6092458\audio...


In [173]:
import re

# Define the pattern to match the prefix
pattern = r'c:/machine learning/vp-ws/mpower/voice-data/'

# Use the `str.contains` method to check if the values in the column matches the pattern
mask = df['file_name'].str.contains(pattern)

# Use the `sum` function to count the number of rows that match the pattern
count = mask.sum()
count

64400

In [174]:
df = df[mask] #deleting rows that contain lost files

In [189]:
# Define the prefix to remove
prefix = 'c:/machine learning/vp-ws/mpower/voice-data/'
df = df.apply(lambda s: s.str.replace(prefix, '', regex=True))
suffix = '.tmp'
df = df.apply(lambda s: s.str.replace(suffix, '', regex=True))

In [194]:
df.head()

,file_handle,file_name
600,5392732,audio_audio.m4a-5371dd55-0099-4a63-b6a8-2fd81d...
601,5391081,audio_audio.m4a-8b104d88-57a3-4757-abe0-ab00ea...
602,5388193,audio_audio.m4a-2750d6be-f15f-4615-9f3f-9eab31...
603,5402960,audio_audio.m4a-136c4ee0-3654-4689-b5fe-e986e6...
604,5405269,audio_audio.m4a-73f1c8af-a0cc-4f1a-a951-bf623c...


In [195]:
# Save the dataframe to a CSV file
df.to_csv('voiceMap.csv', index=False)

# converting tmp to wav

In [1]:
from pydub import AudioSegment
import glob
import os

In [5]:
folder=r'C:\Machine Learning\vp-ws\mPower\voice-data'
if not os.path.exists('wav_files'):
    os.makedirs('wav_files')

In [ ]:
for file in os.listdir(folder):
    try:

        audio = AudioSegment.from_file(folder + '/' + file)

        wav_file = file[:-3] + 'wav'  # Replace the '.tmp' extension with '.wav'
        audio.export('wav_files/' + wav_file, format='wav')
    except Exception as e:
        print(f'Error converting {file}: {e}')

# merging the tables

In [49]:
df_d = pd.read_csv('diagnosis.csv')
df_e = pd.read_csv('else.csv')
df_map = pd.read_csv('voiceMap.csv')

In [50]:
df_merged = pd.merge(df_d, df_e, on='healthCode')

In [51]:
df_merged.head(1)

,ROW_ID_x,ROW_VERSION_x,healthCode,professional-diagnosis,ROW_ID_y,ROW_VERSION_y,recordId,createdOn,appVersion,phoneInfo,audio_audio.m4a,audio_countdown.m4a,medTimepoint
0,20580,21,639e8a78-3631-4231-bda1-c911c1b169e5,False,72137,9,177d6c17-6635-4884-8481-a671fca06266,1425904701000,"version 1.0, build 7",iPhone 6,5404521,5404533,I don't take Parkinson medications


In [52]:
df_merged.rename(columns={'audio_audio.m4a': 'file_handle'},inplace=True)

In [53]:
df_merged.head(1)

,ROW_ID_x,ROW_VERSION_x,healthCode,professional-diagnosis,ROW_ID_y,ROW_VERSION_y,recordId,createdOn,appVersion,phoneInfo,file_handle,audio_countdown.m4a,medTimepoint
0,20580,21,639e8a78-3631-4231-bda1-c911c1b169e5,False,72137,9,177d6c17-6635-4884-8481-a671fca06266,1425904701000,"version 1.0, build 7",iPhone 6,5404521,5404533,I don't take Parkinson medications


In [54]:
df_merged = pd.merge(df_merged, df_map, on='file_handle')

In [55]:
df_merged.head(1)

,ROW_ID_x,ROW_VERSION_x,healthCode,professional-diagnosis,ROW_ID_y,ROW_VERSION_y,recordId,createdOn,appVersion,phoneInfo,file_handle,audio_countdown.m4a,medTimepoint,file_name
0,20582,21,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,False,83210,10,5c3b56ea-2a8a-4a9b-9d04-271928ff7044,1426297608000,"version 1.0, build 7",iPhone 5s (GSM),5773552,5773599,I don't take Parkinson medications,audio_audio.m4a-b68fe4c0-db31-4b39-8f8f-7fb874...


clean the data

In [56]:
df_merged.shape

(62717, 14)

In [57]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62717 entries, 0 to 62716
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ROW_ID_x                62717 non-null  int64 
 1   ROW_VERSION_x           62717 non-null  int64 
 2   healthCode              62717 non-null  object
 3   professional-diagnosis  62315 non-null  object
 4   ROW_ID_y                62717 non-null  int64 
 5   ROW_VERSION_y           62717 non-null  int64 
 6   recordId                62717 non-null  object
 7   createdOn               62717 non-null  int64 
 8   appVersion              62717 non-null  object
 9   phoneInfo               62717 non-null  object
 10  file_handle             62717 non-null  int64 
 11  audio_countdown.m4a     62717 non-null  int64 
 12  medTimepoint            62495 non-null  object
 13  file_name               62717 non-null  object
dtypes: int64(7), object(7)
memory usage: 7.2+ MB


In [58]:
df_merged['professional-diagnosis'].unique()

array([False, True, nan], dtype=object)

In [59]:
df_merged.isnull().sum()

ROW_ID_x                    0
ROW_VERSION_x               0
healthCode                  0
professional-diagnosis    402
ROW_ID_y                    0
ROW_VERSION_y               0
recordId                    0
createdOn                   0
appVersion                  0
phoneInfo                   0
file_handle                 0
audio_countdown.m4a         0
medTimepoint              222
file_name                   0
dtype: int64

In [60]:
df_merged = df_merged.dropna()

In [61]:
df_merged.shape

(62096, 14)

In [62]:
# number of subjects
df_merged['healthCode'].nunique()

4763

In [63]:
df_merged['professional-diagnosis']=df_merged['professional-diagnosis'].astype(bool)

In [64]:
# number of subjects with PD
df_merged['professional-diagnosis'].eq(True).sum()

39283

In [65]:
# number of control subjects
df_merged['professional-diagnosis'].eq(False).sum()

22813

In [66]:
df_merged.head()

,ROW_ID_x,ROW_VERSION_x,healthCode,professional-diagnosis,ROW_ID_y,ROW_VERSION_y,recordId,createdOn,appVersion,phoneInfo,file_handle,audio_countdown.m4a,medTimepoint,file_name
0,20582,21,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,False,83210,10,5c3b56ea-2a8a-4a9b-9d04-271928ff7044,1426297608000,"version 1.0, build 7",iPhone 5s (GSM),5773552,5773599,I don't take Parkinson medications,audio_audio.m4a-b68fe4c0-db31-4b39-8f8f-7fb874...
1,20582,21,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,False,83211,10,9a3d4bd0-a6a6-4218-a442-4bdf4785e953,1426297632000,"version 1.0, build 7",iPhone 5s (GSM),5805736,5805781,I don't take Parkinson medications,audio_audio.m4a-3e719e45-187b-4cbb-8ce1-8ccf37...
2,20594,21,26ce0a95-9946-49f5-b3db-2f095ac9fcd4,False,72815,9,e9fb4ed1-0b62-4ac4-a271-b8b140b696e7,1425939825000,"version 1.0, build 7",iPhone 6,5400756,5400772,I don't take Parkinson medications,audio_audio.m4a-579f1a37-3edd-4df7-b80b-bffc14...
3,20595,21,a9b28b95-b9ee-4442-98c9-1cf9b8cf4a36,False,76066,9,093c7c85-6476-49e1-8047-4823d47a5356,1426015917000,"version 1.0, build 7",iPhone 6 Plus,5565223,5565307,I don't take Parkinson medications,audio_audio.m4a-eeee8102-596c-4030-8f77-b40460...
4,20595,21,a9b28b95-b9ee-4442-98c9-1cf9b8cf4a36,False,76068,9,43ca6e05-be58-4e13-84f3-1536ef491a7e,1426015942000,"version 1.0, build 7",iPhone 6 Plus,5458885,5458962,I don't take Parkinson medications,audio_audio.m4a-4374d36b-4020-4e7a-860b-44aadf...


In [68]:
df_merged.to_csv('merged.csv', index=False)

In [71]:
# removing unnecessary columns - leaving a table for the diagnosis of each recording
df = df_merged[['professional-diagnosis', 'file_name']]

In [73]:
df.to_csv('mPower_data.csv', index=False)

# deleting audio-files with no diagnosis

adding '.wav' as a suffix to each file-name

In [9]:
file = 'recordings-diagnosis.csv'
df = pd.read_csv(file)
df['file_name'] = df['file_name'] + '.wav'
df.to_csv(file, index=False)


checking and deleting files that do not exist in the recordings-diagnosis csv

In [ ]:
 # Setting the path to the directory containing the files we want to delete
directory = 'C:\\Machine Learning\\vp-ws\\mPower\\wav_files'

# Setting the path to the CSV file
csv_file = 'C:\\Machine Learning\\vp-ws\\mPower\\recordings-diagnosis.csv'

# Read the CSV file and store the names in a list
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    names = [row[1] for row in reader]
# Iterate over the files in the directory
for filename in os.listdir(directory):
    # Check if the file name is not in the list of names
    if filename not in names:
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Delete the file
        os.remove(file_path)


removing rows of files that do not exist in the folder

In [14]:
# Open the input CSV file and create a CSV reader
with open(csv_file, 'r') as input_csv:
    reader = csv.reader(input_csv)
    
    # Get the column names from the first row of the input CSV file
    column_names = next(reader)
    
    # Open the output CSV file and create a CSV writer
    with open('output.csv', 'w', newline='') as output_csv:
        writer = csv.writer(output_csv)
        
        # Write the column names to the output CSV file
        writer.writerow(column_names)
        
        # Iterate over the rows in the input CSV file
        for row in reader:
            # Check if the file corresponding to the row exists in the directory
            file_path = os.path.join(directory, row[1])
            if os.path.exists(file_path):
                # Write the row to the output CSV file if the file exists
                writer.writerow(row)
